In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import qutip as qt
import xarray as xr

In this notebook we will show how to do multiprocessing on a "real life" example using `qutip`.

## Influence of spin bath on electron Ramsey

In this notebook we are going to simulate the Ramsey signal from an electron spin, where the precession frequency is slightly different for each experiment because of a nuclear spin bath. The nuclear spin bath varies slowly, which means that frequency is stable within a single experiment.

Loosly based on https://nbviewer.org/urls/qutip.org/qutip-tutorials/tutorials-v5/time-evolution/003_qubit-dynamics.ipynb

For this we first want to have a function that can simulate the signal over time of a single experiment for a given detuning of the precession frequency compared to a rotation frame.

In [ ]:
def ramsey(time_array_s, detuning_hz=0):
    H = detuning_hz * 2 * np.pi * qt.sigmaz() / 2.0
    initial_state = 1 / np.sqrt(2) * (qt.basis(2, 0) + qt.basis(2, 1))
    result = qt.mesolve(H, initial_state, time_array_s, e_ops=[qt.sigmax()])
    sigma_x_timetrace = result.expect[0]
    return sigma_x_timetrace

Let's first test that our function works as expected:

In [ ]:
time_array_s = np.linspace(0, 2e-6)
sigma_x = ramsey(time_array_s, 1e6)
plt.plot(time_array_s, sigma_x)
plt.title("Signal of 1 MHz detuning electron spin")
plt.xlabel("Time [s]")
plt.ylabel(r"$\sigma_x$");

To make our notebooks cleaner and our precious simulation function reusable we move the function into our own library: `src/simulation_tutorial/lib.py`

In [ ]:
from simulation_tutorial import lib

lib.ramsey

# Using apply_ufunc

Let's setup the same analysis in `xarray` so that we can easily sweep variables.

In [ ]:
US_TO_S = 1e-6
MHZ_TO_HZ = 1e6

In [ ]:
ds = xr.Dataset()
ds["time_us"] = np.linspace(0, 2, 100)
ds.time_us.attrs = {"long_name": "Time", "units": "us"}
ds["time_s"] = ds.time_us * US_TO_S
ds["detuning_MHz"] = 1
ds.detuning_MHz.attrs = {"long_name": "Detuning", "units": "MHz"}
ds["detuning_Hz"] = ds.detuning_MHz * MHZ_TO_HZ
ds["sigma_x"] = xr.apply_ufunc(
    ramsey,
    ds.time_s,
    ds.detuning_Hz,
    input_core_dims=[["time_us"], []],
    output_core_dims=[["time_us"]],
    vectorize=True,
    keep_attrs=True,
)
ds.sigma_x.attrs = {"long_name": r"$\sigma_x$"}
ds

In [ ]:
ds.sigma_x.plot()

## How would a fluctuation bath show up in a Ramsey measurement

In [ ]:
GAUSSIAN_FWHM_TO_SIGMA = 1 / (
    2 * np.sqrt(2 * np.log(2))
)  # https://en.wikipedia.org/wiki/Full_width_at_half_maximum

In [ ]:
mc_size = 1000
detuning_MHz_mean = 0
detuning_MHz_FWHM = 0.5
detuning_MHz_sigma = detuning_MHz_FWHM * GAUSSIAN_FWHM_TO_SIGMA
ds = xr.Dataset()
ds["time_us"] = np.linspace(0, 2, 100)
ds.time_us.attrs = {"long_name": "Time", "units": "us"}
ds["time_s"] = ds.time_us * US_TO_S
ds["detuning_MHz"] = xr.DataArray(
    np.random.normal(detuning_MHz_mean, detuning_MHz_sigma, mc_size), dims="mc"
)
ds.detuning_MHz.attrs = {"long_name": "Detuning", "units": "MHz"}
ds["detuning_Hz"] = ds.detuning_MHz * MHZ_TO_HZ
ds["sigma_x"] = xr.apply_ufunc(
    ramsey,
    ds.time_s,
    ds.detuning_Hz,
    input_core_dims=[["time_us"], []],
    output_core_dims=[["time_us"]],
    vectorize=True,
    keep_attrs=True,
)
ds.sigma_x.attrs = {"long_name": r"$\sigma_x$"}
ds

Let's inspect the result of the simulations by plotting the first 10 realizations.

In [ ]:
ds.sigma_x.isel(mc=slice(0, 10)).plot(hue="mc");

We can now take the average over the 'mc' dimension to get the expectation value of $\sigma_x$.

In [ ]:
ds["sigma_x_mean"] = ds.sigma_x.mean("mc")
ds.sigma_x_mean.attrs = {"long_name": r"E($\sigma_x$)"}
ds.sigma_x_mean.plot();

## Now lets use multiprocessing

To speed this up we can use multiprocessing. For this I wrote a wrapper around apply_ufunc, which has the same functionality but on top accepts an argument `multiprocessing=True`. This only works when `vectorize=True` as it distributes the separate function calls over all processors. Without vectorizing there is only a single function call and this is usually much faster anyway. If you need it anyway use `dask`, see later in this notebook.

The number of function calls given to a processor at once can be set with the argument `chunksize`.

In [ ]:
from simulation_tutorial import xarray_mods as xrmod

In [ ]:
mc_size = 1000
detuning_MHz_mean = 0
detuning_MHz_FWHM = 0.5
detuning_MHz_sigma = detuning_MHz_FWHM * GAUSSIAN_FWHM_TO_SIGMA
ds = xr.Dataset()
ds["time_us"] = np.linspace(0, 2, 100)
ds.time_us.attrs = {"long_name": "Time", "units": "us"}
ds["time_s"] = ds.time_us * US_TO_S
ds["detuning_Mhz"] = xr.DataArray(
    np.random.normal(detuning_MHz_mean, detuning_MHz_sigma, mc_size), dims="mc"
)
ds.detuning_Mhz.attrs = {"long_name": "Detuning", "units": "MHz"}
ds["detuning_hz"] = ds.detuning_Mhz * MHZ_TO_HZ
ds["sigma_x"] = xrmod.apply_ufunc(
    ramsey,
    ds.time_s,
    ds.detuning_hz,
    input_core_dims=[["time_us"], []],
    output_core_dims=[["time_us"]],
    vectorize=True,
    keep_attrs=True,
    multiprocessing=True,
    chunksize=50,
)
ds.sigma_x.attrs = {"long_name": r"$\sigma_x$"}
ds["sigma_x_exp"] = ds.sigma_x.mean("mc")
ds.sigma_x_exp.attrs = {"long_name": r"$\sigma_x$"}
ds

# Double sweep

Let's now sweep the detuning FWHM as well to understand its effect.

In [ ]:
mc_size = 1000
detuning_MHz_mean = 0
ds = xr.Dataset()
ds["time_us"] = np.linspace(0, 2, 100)
ds.time_us.attrs = {"long_name": "Time", "units": "us"}
ds["time_s"] = ds.time_us * US_TO_S
ds["detuning_MHz_FWHM"] = np.arange(0.2, 0.71, 0.1)
ds.detuning_MHz_FWHM.attrs = {"long_name": "FWHM", "units": "MHz"}
ds["detuning_MHz_sigma"] = ds.detuning_MHz_FWHM * GAUSSIAN_FWHM_TO_SIGMA
ds["detuning_Mhz"] = xr.apply_ufunc(
    np.random.normal,
    detuning_MHz_mean,
    ds.detuning_MHz_sigma,
    mc_size,
    output_core_dims=[["mc"]],
    vectorize=True,
)
ds.detuning_Mhz.attrs = {"long_name": "Detuning", "units": "MHz"}
ds["detuning_hz"] = ds.detuning_Mhz * MHZ_TO_HZ
ds["sigma_x"] = xrmod.apply_ufunc(
    ramsey,
    ds.time_s,
    ds.detuning_hz,
    input_core_dims=[["time_us"], []],
    output_core_dims=[["time_us"]],
    vectorize=True,
    keep_attrs=True,
    multiprocessing=True,
    chunksize=50,
)
ds.sigma_x.attrs = {"long_name": r"$\sigma_x$"}
ds["sigma_x_exp"] = ds.sigma_x.mean("mc")
ds.sigma_x_exp.attrs = {"long_name": r"$\sigma_x$"}
ds

In [ ]:
ds.sigma_x_exp.plot(hue="detuning_MHz_FWHM");

## Saving to HDF5

For long simulations it is worth to save the results to disk. The big advantage of `xarray` is that saving to dist in HDF5 format is very easy:

In [ ]:
ds.to_netcdf("../data/ramsey.hdf5", engine="h5netcdf")

Also opening it again and using it for plotting is easy. 

For long simulations it is good practice to save the results to disk after simulation and read them from disk before plotting. In this way you don't lose all computation if the kernel crashes during plotting.

In [ ]:
ds_loaded = xr.load_dataset("../data/ramsey.hdf5")
ds_loaded.sigma_x_exp.plot(hue="detuning_MHz_FWHM");

## Multiprocessing using dask

`xarray` offers also a native way of multiprocessing using `dask`. It works less intuitive then my wrapper and has a much larger overhead so usually is slower that my wrapper, but allows for much more complicated features such as, larger than memory arrays, lazy evaluation, multithreading and multiprocessing and distributed computing.

To speed this up we can use `dask` arrays which can do multiprocessing.
For this we need to chuck the data in parts that we want to run at once, e.g.


In [ ]:
ds.detuning_hz.chunk(50)

In [ ]:
from dask.diagnostics import ProgressBar

You cannot call the `chunk` directly on a coordinate of a dataset. Therefore you first need to pass it to `xr.DataArray`

In [ ]:
number_of_mc_realizations = 1000
detuning_MHz_mean = 0
detuning_MHz_sigma = 0.5
ds = xr.Dataset()
ds["time_us"] = np.linspace(0, 2, 100)
ds.time_us.attrs = {"long_name": "Time", "units": "us"}
ds["time_s"] = ds.time_us * US_TO_S
ds["sigma_MHz"] = np.arange(0.2, 0.82, 0.1)
ds.sigma_MHz.attrs = {"long_name": "Sigma", "units": "MHz"}
ds["detuning_Mhz"] = xr.apply_ufunc(
    np.random.normal,
    detuning_MHz_mean,
    ds.sigma_MHz,
    number_of_mc_realizations,
    output_core_dims=[["mc"]],
    vectorize=True,
)
ds.detuning_Mhz.attrs = {"long_name": "Detuning", "units": "MHz"}
ds["detuning_hz"] = ds.detuning_Mhz * MHZ_TO_HZ
ds["sigma_x"] = xrmod.apply_ufunc(
    ramsey,
    ds.time_s,
    xr.DataArray(ds.detuning_hz).chunk(50),
    input_core_dims=[["time_us"], []],
    output_core_dims=[["time_us"]],
    vectorize=True,
    keep_attrs=True,
    dask="parallelized",
)
ds.sigma_x.attrs = {"long_name": r"$\sigma_x$"}
ds["sigma_x_exp"] = ds.sigma_x.mean("mc")
ds.sigma_x_exp.attrs = {"long_name": r"$\sigma_x$"}
with ProgressBar():
    ds = ds.compute(scheduler="processes")
ds

Which is faster than running it on a single core, but not as fast as my wrapper.